In [ ]:
import numpy as np
import cv2
class BackgroundModel:
    def __init__(self, width, height, num_clusters=5, manhattan_threshold=10, L=512):
        self.width = width
        self.height = height
        self.num_clusters = num_clusters
        self.manhattan_threshold = manhattan_threshold
        self.L = L

        # Initialize clusters: (weight, Y, Cb, Cr)
        self.clusters = np.zeros((height, width, num_clusters, 4))  # (weight, Y, Cb, Cr)

    def manhattan_distance(self, centroid, pixel):
        """Compute Manhattan distance between a centroid and a pixel."""
        Y, Cb, Cr = centroid[1:]  # Centroid's components
        p_Y, p_Cb, p_Cr = pixel   # Pixel components
        return abs(Y - p_Y) + abs(Cb - p_Cb) + abs(Cr - p_Cr)

    def update_weights(self, weights, matched_index):
        """Update weights."""
        for k in range(len(weights)):
            if k == matched_index:
                weights[k] += (1 / self.L) * (1 - weights[k])
            else:
                weights[k] += (1 / self.L) * (0 - weights[k])

    def normalize_weights(self, weights):
        """Normalize weights to sum to 1."""
        total = np.sum(weights)
        return weights / total if total > 0 else weights

    def adapt_centroid(self, centroid, pixel):
        """Adapt the centroid towards the incoming pixel."""
        error = centroid[1:] - pixel
        overflow = error > self.L - 1
        underflow = error < -self.L
        adjustment = np.where(overflow, -1, np.where(underflow, 1, 0))
        centroid[1:] += adjustment

    def classify_pixel(self, cluster_weights, matched_index):
        """Classify a pixel as foreground or background."""
        P = np.sum(cluster_weights[matched_index + 1:])
        return P

    def process_frame(self, frame):
        """Process a single frame."""
        output = np.zeros((self.height, self.width), dtype=np.uint8)

        for y in range(self.height):
            for x in range(self.width):
                pixel = frame[y, x]
                cluster_group = self.clusters[y, x]

                # Step 1: Cluster matching
                distances = [
                    self.manhattan_distance(cluster, pixel)
                    for cluster in cluster_group
                ]
                matches = [i for i, d in enumerate(distances) if d <= self.manhattan_threshold]

                if matches:
                    # Matching cluster found
                    matched_index = matches[0]
                    self.adapt_centroid(cluster_group[matched_index], pixel)
                    self.update_weights(cluster_group[:, 0], matched_index)
                else:
                    # No matching cluster found
                    min_weight_index = np.argmin(cluster_group[:, 0])
                    cluster_group[min_weight_index] = np.array([0.01, *pixel])

                # Normalize weights
                cluster_group[:, 0] = self.normalize_weights(cluster_group[:, 0])

                # Sort clusters by weight
                cluster_group = cluster_group[np.argsort(cluster_group[:, 0])[::-1]]

                # Classification
                P = self.classify_pixel(cluster_group[:, 0], matches[0] if matches else -1)
                output[y, x] = 255 if P > 0.5 else 0  # Binary foreground/background

                # Save back sorted clusters
                self.clusters[y, x] = cluster_group

        return output



In [10]:
import os
def main():
    # Open a video file or capture from a camera
    cap = cv2.VideoCapture(r'C:\Users\muxia\Desktop\ece420final\sample4.mp4')  # Replace 'video.mp4' with 0 for live camera

    # Check if video capture is successful
    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        exit()

    # Define target dimensions
    target_width = 320
    target_height = 240

    # Initialize BackgroundModel with resized frame dimensions
    model = BackgroundModel(target_width, target_height)

    # Create a directory to store the foreground images
    output_dir = "foreground_frames1"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    frame_count = 0  # To track the frame number

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if(frame_count%2 != 0):
            frame_count += 1
            continue
        # Resize the frame to 320x240

        frame_resized = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
        # Convert the resized frame to YCbCr format
        frame_ycbcr = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2YCrCb)

        # Process frame
        foreground = model.process_frame(frame_ycbcr)

        # Save the foreground image to a file
        output_path = os.path.join(output_dir, f"foreground_{frame_count:04d}.png")
        cv2.imwrite(output_path, foreground)

        print(f"Saved foreground frame: {output_path}")
        frame_count += 1

        # Break on 'q' key press (optional if running interactively)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    print("Processing complete. Foreground frames saved.")
    
if __name__ == "__main__":
    main()

Saved foreground frame: foreground_frames1\foreground_0000.png
Saved foreground frame: foreground_frames1\foreground_0002.png
Saved foreground frame: foreground_frames1\foreground_0004.png
Saved foreground frame: foreground_frames1\foreground_0006.png
Saved foreground frame: foreground_frames1\foreground_0008.png


KeyboardInterrupt: 

In [ ]:
import numpy as np
import cv2
import os

class BackgroundModel:
    def __init__(self, width, height, num_clusters=5, manhattan_threshold=5, L=512):
        self.width = width
        self.height = height
        self.num_clusters = num_clusters
        self.manhattan_threshold = manhattan_threshold
        self.L = L
        self.clusters = np.zeros((height, width, num_clusters, 4))  # (weight, Y, Cb, Cr)

    def manhattan_distance(self, centroid, pixel):
        Y, Cb, Cr = centroid[1:]
        p_Y, p_Cb, p_Cr = pixel
        return abs(Y - p_Y) + abs(Cb - p_Cb) + abs(Cr - p_Cr)

    def update_weights(self, weights, matched_index):
        for k in range(len(weights)):
            if k == matched_index:
                weights[k] += (1 / self.L) * (1 - weights[k])
            else:
                weights[k] += (1 / self.L) * (0 - weights[k])

    def normalize_weights(self, weights):
        total = np.sum(weights)
        return weights / total if total > 0 else weights

    def adapt_centroid(self, centroid, pixel):
        error = centroid[1:] - pixel
        overflow = error > self.L - 1
        underflow = error < -self.L
        adjustment = np.where(overflow, -1, np.where(underflow, 1, 0))
        centroid[1:] += adjustment

    def classify_pixel(self, cluster_weights, matched_index):
        P = np.sum(cluster_weights[matched_index + 1:])
        return P

    def process_frame(self, frame):
        output = np.zeros((self.height, self.width), dtype=np.uint8)
        for y in range(self.height):
            for x in range(self.width):
                pixel = frame[y, x]
                cluster_group = self.clusters[y, x]
                distances = [self.manhattan_distance(cluster, pixel) for cluster in cluster_group]
                matches = [i for i, d in enumerate(distances) if d <= self.manhattan_threshold]
                if matches:
                    matched_index = matches[0]
                    self.adapt_centroid(cluster_group[matched_index], pixel)
                    self.update_weights(cluster_group[:, 0], matched_index)
                else:
                    min_weight_index = np.argmin(cluster_group[:, 0])
                    cluster_group[min_weight_index] = np.array([0.01, *pixel])
                cluster_group[:, 0] = self.normalize_weights(cluster_group[:, 0])
                cluster_group = cluster_group[np.argsort(cluster_group[:, 0])[::-1]]
                P = self.classify_pixel(cluster_group[:, 0], matches[0] if matches else -1)
                output[y, x] = 255 if P > 0.5 else 0
                self.clusters[y, x] = cluster_group
        return output

def postprocess_foreground(foreground, area_threshold=10):
    """Apply post-processing to remove false positives and fill holes."""
    # Morphological opening to remove small noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    clean_foreground = cv2.morphologyEx(foreground, cv2.MORPH_OPEN, kernel)
    
    # Connected components to eliminate small regions
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(clean_foreground, connectivity=8)
    for i in range(1, num_labels):  # Skip the background (label 0)
        if stats[i, cv2.CC_STAT_AREA] < area_threshold:
            clean_foreground[labels == i] = 0

    # Fill holes in the remaining regions
    filled_foreground = cv2.morphologyEx(clean_foreground, cv2.MORPH_CLOSE, kernel)
    return filled_foreground


In [ ]:
cap = cv2.VideoCapture(r'C:\Users\muxia\Desktop\ece420final\sample4.mp4')
ret, frame = cap.read()
if not ret:
    print("Failed to read video")
    exit()

target_width = 320
target_height = 240
model = BackgroundModel(target_width, target_height)

output_dir = "foreground_frames"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_count % 5 != 0:
        frame_count += 1
        continue
    frame_resized = cv2.resize(frame, (target_width, target_height))
    frame_ycbcr = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2YCrCb)
    foreground = model.process_frame(frame_ycbcr)

    # Apply postprocessing
    processed_foreground = postprocess_foreground(foreground)

    output_path = os.path.join(output_dir, f"foreground_{frame_count:04d}.png")
    cv2.imwrite(output_path, processed_foreground)
    print(f"Saved foreground frame: {output_path}")
    frame_count += 1
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
print("Processing complete. Foreground frames saved.")

Saved foreground frame: foreground_frames\foreground_0000.png
Saved foreground frame: foreground_frames\foreground_0005.png
Saved foreground frame: foreground_frames\foreground_0010.png
Saved foreground frame: foreground_frames\foreground_0015.png
Saved foreground frame: foreground_frames\foreground_0020.png
Saved foreground frame: foreground_frames\foreground_0025.png
Saved foreground frame: foreground_frames\foreground_0030.png
Saved foreground frame: foreground_frames\foreground_0035.png
Saved foreground frame: foreground_frames\foreground_0040.png
Saved foreground frame: foreground_frames\foreground_0045.png
Saved foreground frame: foreground_frames\foreground_0050.png
Saved foreground frame: foreground_frames\foreground_0055.png
Saved foreground frame: foreground_frames\foreground_0060.png
Saved foreground frame: foreground_frames\foreground_0065.png
Saved foreground frame: foreground_frames\foreground_0070.png
Saved foreground frame: foreground_frames\foreground_0075.png
Saved fo